# Decorators and Object-Oriented Programming

## Class variables vs. Instance variables
* variables set outside `__init__` belong to the *class* (as opposed to the *instance*) and are shared by all instances of the class
    * these variables can be accessed via __`ClassName.var`__ and __`classInstance.var`__
* variables created inside `__init__` (and all other method functions) and prefaced with __`self.`__ belong to the object *instance* and cannot be accessed via __`ClassName.`__

In [1]:
class Person(object):
    species = 'Human'
    
    def __init__(self, name):
        self.name = name
        print(
            "{}'s species is {}".format(
            self.name, self.species))

In [2]:
p1 = Person('Godzilla')

Godzilla's species is Human


In [3]:
Person.species, p1.species, p1.name

('Human', 'Human', 'Godzilla')

In [4]:
Person.name

AttributeError: type object 'Person' has no attribute 'name'

In [5]:
p2 = Person('Mothra')
p2.name, p2.species

Mothra's species is Human


('Mothra', 'Human')

In [6]:
Person.species = 'animal'

In [7]:
p1.species, p2.species, Person.species

('animal', 'animal', 'animal')

In [8]:
p1.species = 'monster'

In [9]:
Person.species, p1.species, p2.species

('animal', 'monster', 'animal')

## Using Decorators with OOP

In [20]:
class Duck(object):
    def __init__(self, name):
        self._hidden_name = name
        
    def get_name(self):
        '''getter for name attribute'''
        print('Inside the getter')
        return self._hidden_name

    def set_name(self, val):
        '''setter for name attribute'''
        print('Inside the setter')
        self._hidden_name = val
    
    # the property() function returns a special descriptor object
    name = property(get_name, set_name)

In [13]:
d = Duck("Daffy")
d.name

Inside the getter


'Daffy'

In [14]:
d.name = 'Donald'

Inside the setter


In [15]:
d.name

Inside the getter


'Donald'

In [16]:
fowl = Duck('Donald')
fowl.name = 'foo'

Inside the setter


In [17]:
fowl.get_name()

Inside the getter


'foo'

In [18]:
fowl.name

Inside the getter


'foo'

In [19]:
fowl.name = 'Daffy'
fowl.name

Inside the setter
Inside the getter


'Daffy'

In [23]:
class Duck(object):
    def __init__(self, name):
        self._hidden_name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        print('Inside the getter')
        return self._hidden_name
    
    #name = property(name)
    
    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        print('Inside the setter')
        self._hidden_name = val
    

In [24]:
class Duck(object):
    def __init__(self, name):
        self._hidden_name = name
        
    def name(self):
        '''getter for name attribute'''
        print('Inside the getter')
        return self._hidden_name
    name = property(name)
    
    _t0 = name.setter
    
    def name(self, val):
        '''setter for name attribute'''
        print('Inside the setter')
        self._hidden_name = val
    name = _t0(name)

In [25]:
fowl = Duck('Donald')
fowl.name # we no longer have get_name and set_name functions

Inside the getter


'Donald'

In [28]:
# but hidden_name can still be accessed from outside
fowl.name = 'Rick'
fowl.name

Inside the setter
Inside the getter


'Rick'

In [29]:
class Rect(object):
    
    def __init__(self, w, h):
        self.w, self.h = w, h
        
    @property
    def area(self):
        return self.w * self.h
        
r = Rect(3,4)

In [30]:
r.area

12

In [33]:
r.w = 4
r.area

16

In [34]:
r.area = 5

AttributeError: can't set attribute

In [35]:
class Rect(object):
    
    def __init__(self, w, h):
        self._w, self._h = w, h
        self._area = None
        
    @property
    def w(self):
        return self._w
    @w.setter
    def w(self, value):
        self._w = value
        self._area = None
        
    @property
    def h(self):
        return self._h
    @h.setter
    def h(self, value):
        self._h = value
        self._area = None
        
    @property
    def area(self):
        if self._area is None:
            self._area = self._calc_area()
        return self._area
        
    def _calc_area(self):
        return self._w * self._h
    
r = Rect(3,4)


In [36]:
r.area

12

In [37]:
r.__dict__

{'_w': 3, '_h': 4, '_area': 12}

In [38]:
r.w = 4

In [39]:
r.__dict__

{'_w': 4, '_h': 4, '_area': None}

In [40]:
r.area

16

In [41]:
r.area = 5

AttributeError: can't set attribute

# Static and Class Methods
* static methods are methods that don't operate on an instance of the object and therefore are shared by all instances of the object
* class methods are methods that operate on the class itself, rather than instance of the class

In [43]:
class Duck(object):
    def __init__(self, name):
        # data which is intended to be truly private can be preceeded with "dunder"
        self.__name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self.__name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self.__name = val
    
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        print('-' * len(thing))
        print(thing)
        print('-' * len(thing))
        
    #myprint = staticmethod(myprint)

In [44]:
d = Duck('Donald')
d.myprint('Some thing')

----------
Some thing
----------


In [45]:
Duck.myprint('Some thing via the class')

------------------------
Some thing via the class
------------------------


In [46]:
class Example(object):
    __some_data = 'blah'
    __how_many = 0
    
    def __init__(self, val):
        print('in init for Example')
        self.name = val
#         self.__how_many += 1
        type(self).__how_many += 1 # get from object to class
        # Example.__how_many += 1
   
    def __del__(self):
        type(self).__how_many -= 1
        
    # We can use a static (or class) method to get around
    # a brittle __init__ that doesn't quite do what we want.
    @staticmethod
    def list_init(somelist):
        '''allow me to send in a list, and "flatten" it
        into a string with intervening commas'''
        obj = Example('')
        obj.name = ', '.join(somelist)
        return obj
    
    @classmethod
    def list_init_class(cls, somelist): # or class_ or klass
        self = cls('')
        self.name = ', '.join(somelist)
        return self  

    @classmethod
    def get_some_data(cls):
        return cls.__some_data
    
class Example2(Example):
    '''derived from Example, so we inherit all methods'''
    pass


In [47]:
e = Example(['foo', 'bar', 'baz'])
e.name

in init for Example


['foo', 'bar', 'baz']

In [48]:
e2 = Example.list_init(['foo', 'bar', 'baz'])
e2.name

in init for Example


'foo, bar, baz'

In [49]:
print(type(e), type(e2), e.name, e2.name)

<class '__main__.Example'> <class '__main__.Example'> ['foo', 'bar', 'baz'] foo, bar, baz


In [50]:
e3 = Example2.list_init(['foo', 'bar', 'baz'])
type(e3), e3.name

in init for Example


(__main__.Example, 'foo, bar, baz')

In [51]:
e4 = Example2.list_init_class(['foo', 'bar', 'baz'])
type(e4), e4.name

in init for Example


(__main__.Example2, 'foo, bar, baz')

In [52]:
e = Example(['foo', 'bar', 'baz'])
e2 = Example.list_init(['foo', 'bar', 'baz'])
print(type(e), type(e2), e.name, e2.name)


in init for Example
in init for Example
<class '__main__.Example'> <class '__main__.Example'> ['foo', 'bar', 'baz'] foo, bar, baz


In [53]:
ex2 = Example2('')
ex3 = Example2.list_init(['foo', 'bar'])
type(ex3), Example._Example__how_many


in init for Example
in init for Example


(__main__.Example, 4)

In [54]:
Example2.get_some_data()

'blah'

In [55]:
Example.__dict__

mappingproxy({'__module__': '__main__',
              '_Example__some_data': 'blah',
              '_Example__how_many': 4,
              '__init__': <function __main__.Example.__init__(self, val)>,
              '__del__': <function __main__.Example.__del__(self)>,
              'list_init': <staticmethod at 0x110d4c588>,
              'list_init_class': <classmethod at 0x110d4c978>,
              'get_some_data': <classmethod at 0x110d4ccc0>,
              '__dict__': <attribute '__dict__' of 'Example' objects>,
              '__weakref__': <attribute '__weakref__' of 'Example' objects>,
              '__doc__': None})

In [56]:
class Registered(object):
    _instances = {}
    
    def __init__(self, name):
        self.name = name
        self.register(name, self)
        
    def __repr__(self):
        return '<{} {}>'.format(
            self.__class__.__name__,
            self.name
        )
    
    @classmethod
    def register(cls, name, value):
        cls._instances[name] = value
    
    @classmethod
    def by_name(cls, name):
        return cls._instances[name]
    
foo = Registered('foo')
bar = Registered('bar')

In [57]:
Registered.by_name('foo')

<Registered foo>

In [58]:
Registered.by_name('bar')

<Registered bar>

In [59]:
class Subregistered(Registered):
    pass

baz = Subregistered('baz')
Registered._instances

{'foo': <Registered foo>, 'bar': <Registered bar>, 'baz': <Subregistered baz>}

In [60]:
class PrivateRegistered(Registered):
    _instances = {}
    
bat = PrivateRegistered('bat')

In [61]:
Registered._instances

{'foo': <Registered foo>, 'bar': <Registered bar>, 'baz': <Subregistered baz>}

In [62]:
PrivateRegistered._instances

{'bat': <PrivateRegistered bat>}

### Aside on singletons / borg

In [63]:
class Singleton(object):
    _instance = None
    def __init__(self):
        if self._instance is not None:
            raise ValueError('Singleton!')
        ...
        Singleton._instance = self
    
    @classmethod
    def get(cls):
        if cls._instance is None:
            result = cls()
        else:
            result = cls._instance
        return result

In [64]:
s = Singleton.get()
s2 = Singleton.get()
s is s2

True

In [66]:
class MyBorg(object):
    _state = {}
    def __init__(self):
        self.__dict__ = self._state

In [68]:
b0 = MyBorg()
b1 = MyBorg()
b0.a = 'foo'

In [69]:
b1.a

'foo'

In [70]:
class Selfless(object):
    def method(this, a, b):
        print(f'Called method with {this}, {a}, {b}')
obj = Selfless()

In [71]:
obj.method(1, 2)

Called method with <__main__.Selfless object at 0x110d6b128>, 1, 2


In [81]:
class Foo(object):
    def method(*args):
        print(f'method{args}')
        
    @staticmethod
    def static(*args):
        print(f'static{args}')
        
    @classmethod
    def clsmethod(*args):
        print(f'classmethod{args}')

In [82]:
foo = Foo()
Foo.method()

method()


In [83]:
foo.method()

method(<__main__.Foo object at 0x110d678d0>,)


In [84]:
Foo.static()

static()


In [85]:
foo.static()

static()


In [86]:
Foo.clsmethod()

classmethod(<class '__main__.Foo'>,)


In [87]:
foo.clsmethod()

classmethod(<class '__main__.Foo'>,)


# Lab

Open the [OOP Decorators Lab][oop-decorators-lab]

[oop-decorators-lab]: ./oop-decorators-lab.ipynb